In [7]:
import os
import gzip
import shutil
import pandas as pd
import json
from tqdm import tqdm
import warnings
import datetime

# Suppress all warnings
warnings.filterwarnings("ignore")
# Find records with accuracy differences greater than 50%
def find_significant_differences(df):
    grouped = df.groupby(['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE'])
    significant_differences = grouped.filter(lambda x: (x['ACCURACY'].max() - x['ACCURACY'].min()) / x['ACCURACY'].min() > 0.50)
    return significant_differences


def process_gz_files(directory_path, json_file_path, output_directory, significant_diff_output_directory, num_of_records_output_directory):
    """
    Process .gz files, filter records based on visitor IDs from JSON, and save results.

    Parameters:
    directory_path (str): Path to the directory containing .gz files.
    json_file_path (str): Path to the JSON file containing a list of visitor IDs.
    output_directory (str): Directory to save the filtered CSV files.
    significant_diff_output_directory (str): Directory to save the records with significant accuracy differences.
    """
    # Create the output directories if they don't exist
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(significant_diff_output_directory, exist_ok=True)

    # Load visitor IDs from JSON file and convert to a set
    with open(json_file_path, 'r') as f:
        visitors = set(json.load(f))
    visitors= set(visitors)
    print(f'Number of visitors: {len(visitors)}')
    

    # Initialize dictionary to count the number of records in each file
    number_of_rec= {}


    # Initialize an empty DataFrame to accumulate filtered records and significant_difference_records
    accumulated_filtered_df = pd.DataFrame()
    accumulated_significant_diff_df = pd.DataFrame()

    # Walk through all subdirectories and files
    gz_files = []
    for root, dirs, files in os.walk(directory_path):
        gz_files.extend([os.path.join(root, file) for file in files if file.endswith('.gz')])
    
    print(f'No. of .gz files found: {len(gz_files)}')
    # file_number=0
    # Process each .gz file using tqdm to show progress
    for gz_file_path in tqdm(gz_files, desc="Processing .gz files", unit="file"):
        file = os.path.basename(gz_file_path)
        # print(f'Processing file: {gz_file_path}')

        # Unzip the .gz file to extract the CSV
        csv_filename = file.replace('.gz', '')
        # print(f'csv_file: {csv_filename}')
        output_csv_path = os.path.join(output_directory, csv_filename)

        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_csv_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        # Load the extracted CSV into a DataFrame
        df = pd.read_csv(output_csv_path)
        number_of_rec[output_csv_path]= len(df)
        number_of_rec_path = os.path.join(num_of_records_output_directory, 'number_of_rec_per_file.json')
        with open(number_of_rec_path, 'w') as f:
            json.dump(number_of_rec,f)
        # Filter the DataFrame based on visitor IDs
        filtered_df = df[df['DEVICE_ID'].isin(visitors)]
        
        # timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        csv_filename= csv_filename.split('.')[0]

        significant_diff_df = find_significant_differences(filtered_df)
        significant_diff_df = significant_diff_df.sort_values(by =['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE', 'ACCURACY'])
        # accumulated_significant_diff_df = pd.concat([accumulated_significant_diff_df, significant_diff_df], ignore_index=True)
        # Save the significant differences DataFrame to a CSV file
        significant_diff_csv_path = os.path.join(significant_diff_output_directory, f'significant_accuracy_differences_{csv_filename}.csv')
        significant_diff_df.to_csv(significant_diff_csv_path, index=False)
        filtered_df = filtered_df.drop_duplicates(subset=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE', 'ACCURACY'], keep='first')

        # Append the filtered records to the accumulated DataFrame
        # accumulated_filtered_df = pd.concat([accumulated_filtered_df, filtered_df], ignore_index=True)
        # accumulated_filtered_df = accumulated_filtered_df.drop_duplicates(subset=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE', 'ACCURACY'], keep='first')
        # Delete the unzipped CSV file to free space
        os.remove(output_csv_path)

        # Save the accumulated filtered DataFrame to a CSV file
        filtered_csv_path = os.path.join(output_directory, f'filtered_records_{csv_filename}.csv')
        filtered_df.to_csv(filtered_csv_path, index=False)

        # print(f"Processed {file} and added filtered data to accumulated DataFrame")
    
    
    print(f"Final accumulated data saved to {filtered_csv_path}")


    print(f"Records with significant accuracy differences saved to {significant_diff_csv_path}")

    print(f' Count of records per file saved as {number_of_rec_path}')

    return accumulated_filtered_df, accumulated_significant_diff_df

# Example usage
directory_path = r'C:\Users\Admin\Desktop\Data'
json_file_path = r'C:\Users\User\Downloads\mobility_traces_complete_data\JSON files for visitors IDs_ first_run_correct_one\List of Visitors\Distinct_visitors_upto-600.json'
output_directory = r'C:\Users\User\Downloads\mobility_traces_complete_data\filtered_records'
significant_diff_output_directory = r'C:\Users\User\Downloads\mobility_traces_complete_data\duplicate_records_with_50_pcnt_diff_accuracy'
num_of_records_output_directory = r'C:\Users\User\Downloads\mobility_traces_complete_data\Number of records per file'
accumulated_filtered_df, significant_diff_df = process_gz_files(directory_path, json_file_path, output_directory, significant_diff_output_directory, num_of_records_output_directory)


Number of visitors: 388114
No. of .gz files found: 601


Processing .gz files: 100%|██████████| 601/601 [3:33:11<00:00, 21.28s/file]  


Final accumulated data saved to C:\Users\User\Downloads\mobility_traces_complete_data\filtered_records\filtered_records_data_1_1_9.csv
Records with significant accuracy differences saved to C:\Users\User\Downloads\mobility_traces_complete_data\duplicate_records_with_50_pcnt_diff_accuracy\significant_accuracy_differences_data_1_1_9.csv
 Count of records per file saved as C:\Users\User\Downloads\mobility_traces_complete_data\Number of records per file\number_of_rec_per_file.json


In [1]:
accumulated_filtered_df.columns

NameError: name 'accumulated_filtered_df' is not defined

In [23]:
# Identify duplicate records based on specific columns
accumulated_filtered_df = accumulated_filtered_df.sort_values(by=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE'])
duplicates = accumulated_filtered_df[accumulated_filtered_df.duplicated(subset=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE'], keep=False)]
duplicates.shape # 128503 on basis of id, datetime, lat, lon
# duplicates.shape # (97464, 11)  on basis of id, datetime, lat, lon, accuracy
duplicates.head

(128503, 11)

In [44]:
accumulated_filtered_df['DEVICE_ID'].nunique()

88011

In [ ]:
accumulated_filtered_df_with_unique_accuracy = accumulated_filtered_df.drop_duplicates(subset=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE', 'ACCURACY'], keep='first')
duplicates = accumulated_filtered_df_with_unique_accuracy[accumulated_filtered_df_with_unique_accuracy.duplicated(subset=['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE'], keep=False)]
duplicates.shape #39787
duplicates.head(25)

In [45]:
duplicates['DEVICE_ID'].nunique()

10694

In [46]:
# duplicates= duplicates.reset_index()

# Group by DEVICE_ID, EVENT_DATE, LATITUDE, LONGITUDE
grouped = duplicates.groupby(['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE'])
print(f' number of groups : {len(grouped)}')
# Function to calculate if there is a difference of more than 10% in accuracy
def has_significant_difference(group):
    if len(group) < 2:
        return False
    max_accuracy = group['ACCURACY'].max()
    min_accuracy = group['ACCURACY'].min()
    return (max_accuracy - min_accuracy) / min_accuracy > 0.1

# Apply the function and filter groups with significant difference
significant_differences = grouped.filter(has_significant_difference)

# Get the count of such groups
count_of_significant_differences = len(significant_differences.groupby(['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE']))

# Display result
print(f"Number of groups with more than 10% difference in accuracy: {count_of_significant_differences}")


 number of groups : 19336
Number of groups with more than 10% difference in accuracy: 5359


In [36]:
# Filter groups with significant difference
significant_differences = grouped.filter(has_significant_difference)

# Display the records with significant differences in accuracy
# print("Records with more than 10% difference in accuracy:")
# print(significant_differences)

# If you want to list the accuracy values by group
for name, group in significant_differences.groupby(['DEVICE_ID', 'EVENT_DATE', 'LATITUDE', 'LONGITUDE']):
    print(f"Group: {name}")
    print(group[['ACCURACY']])
    print('-' * 40)

Group: ('07e242d1-26b7-40f2-9a35-9a902a2523e5', '2022-09-01 02:45:09.000', np.float64(29.84074), np.float64(79.77302))
      ACCURACY
1240       1.0
1241      13.5
----------------------------------------
Group: ('0cc8a2fc-8496-4ba6-832e-22b2a9d735ba', '2022-09-01 13:49:53.000', np.float64(28.955687), np.float64(79.42922))
      ACCURACY
2021       1.5
2022      16.6
----------------------------------------
Group: ('23b0cc89-834b-4cde-80ad-765fc0ca91bf', '2022-09-01 06:32:44.000', np.float64(26.0711517), np.float64(76.3649))
      ACCURACY
5478       1.4
5479      16.3
----------------------------------------
Group: ('24d14c25-4c96-425c-b0b4-30809bd83df8', '2022-09-01 10:46:48.000', np.float64(28.482801), np.float64(79.28264))
      ACCURACY
5600       1.0
5601      11.1
----------------------------------------
Group: ('43d4c6e6-ad5b-47aa-a493-4eff1dbeee39', '2022-09-01 18:21:49.000', np.float64(22.30327), np.float64(73.17916))
        ACCURACY
10232   1.378463
10233  16.000000
10234  